Several step 
  - Find all the url that we need to collect data from
  - Collect and clean the data 
  - More cleaning & transform the data into a match up graph (basicly network graph)
  - linear optimisation to find the probability I should be playing the character

Let's First find the url of all figther data we need to extract

In [1]:
# get match up data from sf5 
from BeautifulSoup import BeautifulSoup
import urllib2
import re
import pandas as pd
import numpy as np
import csv

# global variables
MAIN_URL = 'http://www.eventhubs.com'

# read the main url 
html_page = urllib2.urlopen("http://www.eventhubs.com/tiers/sf5/")
soup = BeautifulSoup(html_page)

# extract the fighter url page
url_raw = []
for link in soup.findAll('a'):
    href = link.get('href')
    href = str(href)
    if '/tiers/sf5/character/' in href and 'vote' not in href:
        url_raw.append(href)
        
url_fighter_list = [MAIN_URL + string_in_list for string_in_list in url_raw]

Let's collect the data

In [62]:
# for every fighter url read all table data 
# transform in 3 columns data : fighter_name , fighter_against_name, match_avg_score.

for row,url_fighter in enumerate(url_fighter_list):
    
    html_fighter_page = urllib2.urlopen(url_fighter)
    soup_fighter = BeautifulSoup(html_fighter_page)

    # find fighter opponents name
    fighter_against_list_name = []

    for html_row in soup_fighter.findAll('td'):
        html_row_text = str(html_row) 
        if '/tiers/sf5' in html_row_text:
            start_character_number = html_row_text.index('/tiers/sf5') + 26 
            end_character_number = html_row_text.index('</a>') 
            fighter_against = html_row_text[start_character_number:end_character_number]
            fighter_against_list_name.append(fighter_against)

    # find match avg score for every opponents
    # ad a constraints 3 digits
    match_score_list = []

    for html_row in soup_fighter.findAll('td'):
        html_row_text = str(html_row) 
        match_score_compile = re.search(r"[\d][.][\d]+", html_row_text)

        if match_score_compile is not None:
            match_score_text = match_score_compile.group()
            match_score_text_clean = match_score_text[:3]
            match_score_test_int = int(match_score_text[0]) 

            if match_score_test_int > 0 and len(match_score_text) == 3:
                match_score_list.append(match_score_text_clean)

    # attach fighter_name to data
    fighter_name = url_fighter[url_fighter.index('character')  + 10:]
    fighter_name = np.array([fighter_name])
    repeat_length = len(fighter_against_list_name)
    fighter_name_list = np.repeat(fighter_name, repeat_length, axis=0)

    # prepare the data
    balancing_raw_data = pd.DataFrame([fighter_name_list,fighter_against_list_name, match_score_list])
    balancing_raw_data_transpose = balancing_raw_data.T[0:17]
    balancing_raw_data_transpose.columns = ['fighter_A', 'fighter_B','match_score']
    
    # combine all the data
    if row == 0:
        fighter_data = balancing_raw_data_transpose
    else:
        fighter_data = fighter_data.append(balancing_raw_data_transpose)    
        
# lower fighter name 
fighter_data['fighter_B'] = fighter_data['fighter_B'].str.lower()
fighter_data['fighter_A'] = fighter_data['fighter_A'].str.lower()


Now let's transform the data into a match-up chart

In [246]:
# get unique fighter name (using fighter_A name column)
nrow = len(fighter_data)
figther_name = set()
for row in fighter_data['fighter_A']:
    figther_name.add(row)

# initialize the match up chart
nb_figther = len(figther_name)
tableau = np.zeros([nb_figther,nb_figther])

# fill the matchup chart
# the first two for loop generate all the possible combination of fighter_A and fighter_B
# the output set contain the 18 name of sf5 character 

for i,fighter_A in enumerate(figther_name):
    for j,fighter_B in enumerate(figther_name):
        
        # iterate throught the data x 
        # Find the combination of the fighter_A(i) and fighter_B(j) in the data x
        # Append the the score points to the i and j location in tableau[i,j]
        for row in range(0,nrow):
            if fighter_data.fighter_A.iloc[row] == fighter_A and \
               fighter_data.fighter_B.iloc[row] == fighter_B:
                    tableau[i,j] = fighter_data.match_score.iloc[row]
        
        # if figther_A and fighter_B are the same we assign the value NA in our matrix
        if fighter_A == fighter_B:
            tableau[i,j] = np.nan

# retransform to pandas and ad column and row name
# we can see there is a lot of 0, why is that ? it seems they are not in the table
match_up_chart = pd.DataFrame(tableau)
figther_name = sorted(list(figther_name))

match_up_chart = match_up_chart.set_index([figther_name])
match_up_chart.columns = [figther_name]

match_up_chart

,alex,birdie,cammy,chun-li,dhalsim,fang,guile,karin,ken,laura,m-bison,nash,necalli,r-mika,rashid,ryu,vega,zangief
alex,NaN,5.2,0,5.5,4.9,0,5.4,5.1,4.8,5.1,5.4,5.1,5.0,5.3,5.2,5.1,5.5,5.2
birdie,0,NaN,0,5.3,4.9,0,5.1,5.0,4.4,4.9,5.2,4.9,4.6,4.9,4.8,4.9,5.5,5.0
cammy,0,5.0,NaN,5.1,4.7,0,5.1,5.0,4.3,5.0,5.2,4.9,4.6,4.5,4.4,4.8,4.9,4.9
chun-li,0,4.7,0,NaN,4.2,0,4.9,4.5,4.0,4.6,5.0,4.8,4.6,4.3,4.1,4.4,5.2,5.0
dhalsim,0,5.1,0,5.8,NaN,0,5.4,4.8,4.7,4.7,5.8,5.2,4.8,4.6,4.8,4.6,6.7,5.9
fang,0,4.8,0,5.1,4.2,NaN,4.7,4.1,4.4,4.4,4.8,4.6,4.2,4.5,4.3,4.7,5.8,5.2
guile,0,4.9,0,5.1,4.6,0,NaN,4.6,4.5,4.8,5.1,4.7,4.5,5.2,4.7,4.8,5.8,5.1
karin,0,5.0,0,5.5,5.2,0,5.4,NaN,4.8,4.9,5.3,5.0,4.8,5.3,5.0,5.1,5.0,4.9
ken,0,5.6,0,6.0,5.3,0,5.5,5.2,NaN,5.5,6.1,5.5,5.2,5.0,5.1,5.3,6.5,5.6
laura,0,5.1,0,5.4,5.3,0,5.2,5.1,4.5,NaN,5.1,4.9,4.6,5.1,4.9,5.2,5.0,4.8


In [ ]:
# We see there are some missing value, I will need to investigate this throught

Now let's do linear optimization to identify balancing opportunity.